<a href="https://colab.research.google.com/github/saisriy/ShroudX/blob/main/audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.io import wavfile
from scipy.fft import fft, ifft
from scipy.signal import resample
import cv2
import secrets
import matplotlib.pyplot as plt
import time
from skimage.metrics import structural_similarity as ssim
import os
import math

In [ ]:
def Read_image(image_path) :
    image = cv2.imread(image_path)
    return image

def Write_image(image_path,image) :
    cv2.imwrite(image_path,image)

def Show_image(image,title) :
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()


def resize_image(image_path, output_path, width, height) :
    image = Read_image(image_path)
    resized_image = cv2.resize(image, (width, height))
    Write_image(output_path, resized_image)
    print(f"Resized image saved to {output_path}")


In [ ]:
def audio_to_array(audio_file,quality = 90,downsampling_factor = 20) :

    sample_rate, data = wavfile.read(audio_file)

    if data.ndim > 1 :
      data = data.mean(axis=1)

    frequencyDomainData = fft(data)
    magnitudeSpectrum = np.abs(frequencyDomainData)

    threshold = np.percentile(magnitudeSpectrum, quality)

    dominantData = magnitudeSpectrum > threshold

    filteredData = frequencyDomainData * dominantData

    dominantAudio = np.real(ifft(filteredData))

    downsampled_audio = resample(dominantAudio, int(len(dominantAudio) / downsampling_factor))
    downsampled_audio = np.array(downsampled_audio)
    downsampled_audio = np.int16(downsampled_audio / np.max(np.abs(downsampled_audio)) * 32767)
    downsampled_audio = downsampled_audio / np.max(np.abs(downsampled_audio))
    downsampled_audio = (downsampled_audio + 1) * 127.5
    downsampled_audio = np.uint8(downsampled_audio)

    return downsampled_audio,sample_rate


In [ ]:
def array_to_text(numbers,textfile) :

    with open(textfile, "w") as f:
        f.write(" ".join(map(str, numbers)))


In [ ]:
def text_to_array(textfile) :

    with open(textfile, "r", encoding="utf-8") as file :
        audio_string = file.read()

    audio_array = list(map(int, audio_string.split()))

    return audio_array

In [ ]:
def array_to_audio(audio_file,audio_array,sample_rate,quality = 90,downsampling_factor = 20) :

    audio_array = np.uint8(audio_array)
    wavfile.write(audio_file, int(sample_rate / downsampling_factor), audio_array)

In [ ]:
def are_files_identical(file1_path, file2_path):
    try:
        with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
            # Read both files
            file1_content = file1.read()
            file2_content = file2.read()

            # Compare the contents
            if file1_content == file2_content:
                print("The files are identical.")
                return True
            else:
                print("The files are not identical.")
                return False

    except FileNotFoundError as e:
        print(f"Error: {e}")
        return False

In [ ]:
def embed_text_random_expire(image_path, text_file, output_path, expiry_seconds) :

    def generate_key() :

        return secrets.randbits(32)

    image = cv2.imread(image_path)

    if image is None :

        print(f"Error: Could not read image at path '{image_path}'.")
        return None

    with open(text_file, "r", encoding="utf-8") as file :

        message = list(map(int, file.read().strip().split()))

    timestamp_bin = format(int(time.time()), '032b')
    expiry_bin = format(int(expiry_seconds), '032b')

    end_marker = [255, 255, 255, 255, 255]
    message += end_marker

    binary_message = timestamp_bin + expiry_bin + ''.join(format(num, '08b') for num in message)

    key = generate_key()
    np.random.seed(key)

    if len(binary_message) > image.size // 3:
        print("Attention : Message is too long to be embedded in the image.")
        print("Resizing because of small cover image")

        height, width, _ = image.shape
        resizing_factor = int(math.sqrt(int(int(len(binary_message) / (height * width)) + 1)) + 1)
        new_height = int(height * resizing_factor) + 3
        new_width = int(width * resizing_factor) + 3
        resize_image(image_path,image_path,new_width,new_height)
        image = cv2.imread(image_path)
    pixel_indices = np.random.choice(image.size // 3, len(binary_message), replace=False)

    for i, bit in enumerate(binary_message):
        idx = pixel_indices[i]
        row = idx // image.shape[1]
        col = idx % image.shape[1]
        channel = i % 3
        image[row, col, channel] = (image[row, col, channel] & 254) | int(bit)

    cv2.imwrite(output_path, image)
    print(f"Audio embedded successfully into {output_path}")
    return key


def extract_text_random_expire(image_path, output_text_file, key) :

    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Could not read image at path '{image_path}'.")
        return False

    np.random.seed(key)

    pixel_indices = np.random.choice(image.size // 3, image.size // 3, replace=False)

    binary_bits = []
    for i in pixel_indices:
        row = i // image.shape[1]
        col = i % image.shape[1]
        channel = len(binary_bits) % 3
        bit = image[row, col, channel] & 1
        binary_bits.append(str(bit))


        if len(binary_bits) >= 40 and len(binary_bits) % 8 == 0:
            last_five = [
                int(''.join(binary_bits[-8 * i: -8 * (i - 1) if i > 1 else None]), 2)
                for i in range(1, 6)
            ]
            if last_five == [255, 255, 255, 255, 255]:
                break
    else:
        print("Error: End marker not found. Possibly wrong key or corrupted data.")
        return False

    binary_message = ''.join(binary_bits[:-40])

    try:
        timestamp = int(binary_message[:32], 2)
        expiry = int(binary_message[32:64], 2)
        current = int(time.time())

        if current - timestamp > expiry:
            print("Error: Message has expired and cannot be extracted.")
            return False

        binary_message = binary_message[64:]
        message = [str(int(binary_message[i:i + 8], 2)) for i in range(0, len(binary_message), 8)]

    except Exception as e:
        print(f"Error decoding message: {e}")
        return False

    with open(output_text_file, "w", encoding="utf-8") as file:
        file.write(" ".join(message))

    print(f"Message extracted and saved to {output_text_file}")
    return True

In [ ]:
def Start_Encode(audio_file,Cover_path,expiry,Stego_path) :

    audio_array,sample_rate = audio_to_array(audio_file)

    text_file = 'temporary.txt'
    array_to_text(audio_array,text_file)

    Key = embed_text_random_expire(Cover_path, text_file,Stego_path, expiry)
    print(Key,sample_rate)
    return Key,sample_rate


def Start_Decode(Stego_path,Key,sample_rate,Retrived_path) :

    text_file = 'temporary.txt'
    extract_text_random_expire(Stego_path,text_file,Key)
    audio_array = text_to_array(text_file)
    array_to_audio(Retrived_path,audio_array,sample_rate)

In [ ]:
op,ip = Start_Encode('abcd.wav','256_butter.jpg',100,'stego.png')
Start_Decode('stego.png',op,ip,'Retrived.wav')

Attention : Message is too long to be embedded in the image.
Resizing because of small cover image
Resized image saved to 256_butter.jpg
Audio embedded successfully into stego.png
2995635217 44100
Message extracted and saved to temporary.txt
